In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import torch
from scipy import stats
from lib.DataManager import *
from lib.PolicyStats import *
import os
import cma
from cma.constraints_handler import AugmentedLagrangian, PopulationEvaluator
from IPython import display

# Constants

In [ ]:
# NOTE CHANGE THE NUM_STATES FOR GRIDWORLD / REAL
# num_states = 18
num_states = 23
num_actions = 4
gamma = 0.95
delta = 0.01 #1 - delta, confidence

In [ ]:
histories = GetHistories("gridworld_data.csv")

In [ ]:
avg_exploratory_J = 0
for traj in histories:
    rewards = traj[:,2]
    nonzero_idx = np.where(rewards != 0)[0]
    for idx in nonzero_idx:
        avg_exploratory_J += rewards[idx] * (gamma ** idx)
avg_exploratory_J /= len(histories)
print(avg_exploratory_J)

In [ ]:
split_idx = int(len(histories) * .8)
train = histories[:split_idx]
test = histories[split_idx:]
print(len(train))
print(len(test))

***Get Exploration Policy***

In [ ]:
exploration_policy = GetPolicy(train, num_states, num_actions, 10)
print(exploration_policy)

***Pick Importance Sampling Function***

In [ ]:
# ISFunc = ImportanceSampling
ISFunc = PDImportanceSampling

***Evaluate Current Policy On Candidate/Safety Data***

In [ ]:
# P(Je > J_lower_bound) > 1 - delta
J_predicted_lower_bound = Safety_Prediction(train, exploration_policy, gamma, exploration_policy, ISFunc, delta, len(test))
print(J_predicted_lower_bound)
J_safety_lower_bound = Safety_Test(test, exploration_policy, gamma, exploration_policy, ISFunc, delta)
print(J_safety_lower_bound)

***Helper Functions***

In [ ]:
def policy_softmax(policy):
    numerators = np.exp(policy)
    return (numerators.T / np.sum(numerators, axis=1)).T

In [ ]:
def random_explore():
    best_policy = exploration_policy.copy()
    max_lower_bound = 0

    for i in range(100):
        random_step = np.random.normal(0, 1, best_policy.shape)
        new_policy = policy_softmax(best_policy + random_step)

        J_predicted_lower_bound = Safety_Prediction(train, exploration_policy, gamma, new_policy, ISFunc, delta, len(test))
        print("Predicted Lower Bound: ", J_predicted_lower_bound)
        if(J_predicted_lower_bound > max_lower_bound):
            print("Policy Updated")
            best_policy = new_policy
            max_lower_bound = J_predicted_lower_bound
        print("---------------")

    print(best_policy)
    return best_policy

In [ ]:
def unconstrained_explore():
    def objective(s):
        new_policy = policy_softmax(s.reshape(num_states, num_actions))
        avgIS = CalcAvgIS(train, exploration_policy, gamma, new_policy, ISFunc)
        print(avgIS)
        return - avgIS #minimizing
    
    es = cma.CMAEvolutionStrategy(num_states * num_actions * [0], 0.5)
    while not es.stop():
        solutions = es.ask()
        display.clear_output(True)
        print(policy_softmax(solutions[0].reshape(num_states, num_actions)))
        es.tell(solutions, [objective(s) for s in solutions])
        
    return policy_softmax(es.ask()[0].reshape(num_states, num_actions))

In [ ]:
def inv_barrier_constrained_explore(lower_bound_goal, max_updates=100):
    def constraint(new_policy, avgIS):
        EPSILON = 0.01
        J_predicted_lower_bound = Safety_Prediction(train, exploration_policy, gamma, new_policy, ISFunc, delta, len(test), avgIS)
        return 1 / (max(J_predicted_lower_bound - lower_bound_goal, EPSILON))
    
    def objective(new_policy, avgIS):
        return - avgIS #minimizing
    
    def optimizing_function(s):
        new_policy = policy_softmax(s.reshape(num_states, num_actions))
        avgIS = CalcAvgIS(train, exploration_policy, gamma, new_policy, ISFunc)
        objective_score = objective(new_policy, avgIS)
        constraint_score = constraint(new_policy, avgIS)
        score = objective_score + constraint_score
        print("score : " + str(score) + "\n---constraint_score : " + str(constraint_score) + "\n---objective_score : " + str(objective_score))
        return score
    
    i = 0
    es = cma.CMAEvolutionStrategy(num_states * num_actions * [0], 0.5)
    while (not es.stop() and i != max_updates):
        solutions = es.ask()
        display.clear_output(True)
        print("Update : " + str(i))
        print(policy_softmax(solutions[0].reshape(num_states, num_actions)))
        es.tell(solutions, [optimizing_function(s) for s in solutions])
        i += 1
        
    return es

***Explore Policies***

In [ ]:
# target_performance = 1.41537 #from paper
target_performance = avg_exploratory_J
trained_es = inv_barrier_constrained_explore(target_performance + 0.1, 50)
new_policy = policy_softmax(trained_es.ask()[0].reshape(num_states, num_actions)), 

In [ ]:
print("ES_Convergence : " + str(sum(trained_es.mean**2)))

***Final Results***

In [ ]:
J_predicted_lower_bound = Safety_Prediction(train, exploration_policy, gamma, new_policy, ISFunc, delta, len(test))
print(J_predicted_lower_bound)

In [ ]:
J_safety_lower_bound = Safety_Test(test, exploration_policy, gamma, new_policy, ISFunc, delta)
print(J_safety_lower_bound)

In [ ]:
folder_name = "policies\\delta_" + str(delta) + "\\"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

np.save(folder_name + "safety_" + str(J_safety_lower_bound), new_policy)

In [ ]:
np.argmax(new_policy,axis=1)

In [ ]:
np.save("gridworld" + str(J_safety_lower_bound), new_policy)